# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to /home/freemo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/freemo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("Disaster_Data", engine)
X = df['message']
#drop related as there were records with 3 choices
y = df.drop(['id', 'message', 'original', 'genre', 'related'], axis = 1)
categories = y.columns

In [3]:
categories

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [4]:
# check df
df.to_csv("df.csv", index=True)

In [5]:
# check the shape
X.shape
y.shape

(26216, 35)

Check randomly the two arrays

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """Tokenize and clean text.
    
    Args: message text
    
    Returns: normalized tokens
    """
    # remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ',text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [9]:
# test function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
# split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f08b416f050>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred, labels=[1, 2, 3]))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        39
           2       0.78      0.63      0.70      2722
           3       0.73      0.04      0.07       529

   micro avg       0.78      0.52      0.63      3290
   macro avg       0.50      0.22      0.25      3290
weighted avg       0.76      0.52      0.59      3290
 samples avg       0.26      0.23      0.24      3290



/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5451
           1       0.84      0.40      0.54      1103

    accuracy                           0.89      6554
   macro avg       0.86      0.69      0.74      6554
weighted avg       0.88      0.89      0.87      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

------------------------------------------------------

FEATURE: aid_related

              precision    recall  f1-score   support

           0       0.77      0.87      0.82      3832
           1 

/home/freemo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6433
           1       1.00      0.02      0.05       121

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98      0.98      0.97      6554

------------------------------------------------------

FEATURE: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6500
           1       0.00      0.00      0.00        54

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

------------------------------------------------------

FEATURE: hospitals

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6491
           1       0.00      0.00      0.00        63

    accuracy                      

In [24]:
# display results

# Print classification report on test data
print(classification_report(y_test, y_pred, target_names=y.columns))

                        precision    recall  f1-score   support

               request       0.84      0.40      0.54      1103
                 offer       0.00      0.00      0.00        39
           aid_related       0.78      0.63      0.70      2722
          medical_help       0.73      0.04      0.07       529
      medical_products       0.75      0.04      0.07       342
     search_and_rescue       1.00      0.01      0.01       183
              security       0.50      0.01      0.02       123
              military       0.67      0.01      0.02       218
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.17      0.28       400
                  food       0.87      0.31      0.45       717
               shelter       0.87      0.12      0.21       566
              clothing       0.50      0.03      0.06        88
                 money       0.80      0.02      0.05       162
        missing_people       0.00      

In [25]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

ValueError: Number of classes, 34, does not match size of target_names, 35. Try specifying the labels parameter

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.